In [4]:
import pandas as pd

In [9]:
#!pip install --upgrade tensorflow
#!pip install --upgrade keras

In [10]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.models import Sequential
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping

C:\Users\raunak.sharan\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [11]:
data = pd.read_csv('Data.csv')

In [15]:
data['visits'].loc[0]

'30 84 126 135 137 179 242 342 426 456 460 462 483 594 600 604 704 723 744 787 804 886'

In [16]:
data.set_index('visitor_id', inplace = True)

In [19]:
def get_full_vector(row):
    int_list = [int(x) for x in row.split(' ')]
    ser = pd.Series(data = [0]*910, index = np.arange(1,911,1))
    for x in int_list:
        ser[x] += 1
    return ser

In [20]:
data_full_form = pd.DataFrame(columns = np.arange(1,911,1))

In [22]:
data_dense = data.loc[:,'visits'].apply(func = get_full_vector)

In [115]:
data_dense_copy = data.loc[:,'visits'].apply(func = get_full_vector)

In [26]:
data_dense.head()

,1,2,3,4,5,6,7,8,9,10,...,901,902,903,904,905,906,907,908,909,910
visitor_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
data_test = data_dense.loc[:,904:]

In [103]:
def get_visit_day(row):
    y = np.where(row > 0)[0].tolist()
    day = ['M','Tu','W','Th','F','Sa','Su']
    if(len(y)>0):
        y=day[y[0]]
    else:
        y= 'N'
    return y

def get_visit_day_num(row):
    y = np.where(row > 0)[0].tolist()
    if(len(y)>0):
        y=y[0]+1
    else:
        y= 0
    return y

In [98]:
#data_dense['target'] = data_test.apply(lambda x : np.where(x > 0)[0][0], axis=0)
data_dense['target'] = data_test.apply(func = get_visit_pos, axis=1)

In [99]:
data_dense['target'].value_counts()

N     132260
Sa     29061
M      26878
Su     26094
F      19915
Th     18734
W      18715
Tu     18343
Name: target, dtype: int64

In [104]:
data_dense['label'] = data_test.apply(func = get_visit_day_num, axis=1)

In [106]:
data_dense['label'].value_counts()

0    132260
6     29061
1     26878
7     26094
5     19915
4     18734
3     18715
2     18343
Name: label, dtype: int64

In [107]:
labels = to_categorical(data_dense['label'], num_classes=8)

In [116]:
X = data_dense_copy.loc[:,:903]

In [204]:
l=[]

In [205]:
for j in range(0,129):
    d = X.loc[:,j*7+1:j*7+7].values
    l.append(d)

In [206]:
l

[array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 1]], dtype=int64), array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [1, 0, 0, ..., 1, 0, 0],
        [0, 0, 0, ..., 0, 0, 1],
        [1, 0, 0, ..., 1, 0, 0]], dtype=int64), array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 1, 0, 1]], dtype=int64), array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 1, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 1, 0],
        [0, 0, 0, ..., 0, 1, 1],
        [0, 0, 0, ..., 1, 0, 1]], dtype=int64), array([[0, 1, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
 

In [207]:
X_new = np.array(l)

In [208]:
X_new.shape

(129, 290000, 7)

In [226]:
X_new2 = np.rollaxis(X_new,1)

In [227]:
X_new2.shape

(290000, 129, 7)

In [228]:
#lstm input - input_length*input_dim, input_samples is first dim of input
epochs = 10
batch_size = 256
input_length = X_new2.shape[1]
input_dim = X_new2.shape[2]
output_dim = len(labels[0])

In [229]:
model = Sequential()
model.add(LSTM(4, input_dim = input_dim, input_length = input_length))
model.add(Dense(output_dim, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
'''
model.add(LSTM(units=256, input_shape=(X.shape[0],X.shape[1])))
model.add(Dense(units=903 , activation='softmax'))
'''
print(model.summary())

C:\Users\raunak.sharan\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  
C:\Users\raunak.sharan\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(4, input_shape=(129, 7))`
  


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_18 (LSTM)               (None, 4)                 192       
_________________________________________________________________
dense_17 (Dense)             (None, 8)                 40        
Total params: 232
Trainable params: 232
Non-trainable params: 0
_________________________________________________________________
None


In [230]:
history = model.fit(X_new2, labels, epochs=epochs, batch_size=batch_size,
                    validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',patience=7, min_delta=0.0001)])

Train on 232000 samples, validate on 58000 samples
Epoch 1/10
232000/232000 [==============================] - 193s 834us/step - loss: 1.6961 - acc: 0.4918 - val_loss: 2.0646 - val_acc: 0.2704
Epoch 2/10
232000/232000 [==============================] - 188s 809us/step - loss: 1.6400 - acc: 0.5025 - val_loss: 2.0847 - val_acc: 0.2704
Epoch 3/10
232000/232000 [==============================] - 185s 797us/step - loss: 1.6292 - acc: 0.5025 - val_loss: 2.0898 - val_acc: 0.2704
Epoch 4/10
232000/232000 [==============================] - 189s 814us/step - loss: 1.6250 - acc: 0.5025 - val_loss: 2.0775 - val_acc: 0.2704
Epoch 5/10
232000/232000 [==============================] - 191s 822us/step - loss: 1.6235 - acc: 0.5025 - val_loss: 2.0788 - val_acc: 0.2704
Epoch 6/10
232000/232000 [==============================] - 188s 812us/step - loss: 1.6229 - acc: 0.5025 - val_loss: 2.0831 - val_acc: 0.2704
Epoch 7/10
232000/232000 [==============================] - 185s 799us/step - loss: 1.6197 - acc:

In [ ]:
model = Sequential()
model.add(LSTM(4, input_dim = input_dim, input_length = input_length))
model.add(Dense(output_dim, activation='softmax'))
#model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [235]:
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


model.compile(loss='categorical_crossentropy',
          optimizer= "adam",
          metrics=[f1])

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_18 (LSTM)               (None, 4)                 192       
_________________________________________________________________
dense_17 (Dense)             (None, 8)                 40        
Total params: 232
Trainable params: 232
Non-trainable params: 0
_________________________________________________________________
None


In [236]:
history = model.fit(X_new2, labels, epochs=epochs, batch_size=batch_size,
                    validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',patience=7, min_delta=0.0001)])

Train on 232000 samples, validate on 58000 samples
Epoch 1/10
232000/232000 [==============================] - 183s 788us/step - loss: 1.6142 - f1: 0.3990 - val_loss: 2.0079 - val_f1: 6.8697e-05
Epoch 2/10
232000/232000 [==============================] - 177s 765us/step - loss: 1.6139 - f1: 0.4000 - val_loss: 2.0085 - val_f1: 6.8697e-05
Epoch 3/10
232000/232000 [==============================] - 177s 762us/step - loss: 1.6138 - f1: 0.4025 - val_loss: 2.0073 - val_f1: 2.4044e-04
Epoch 4/10
232000/232000 [==============================] - 178s 768us/step - loss: 1.6137 - f1: 0.4006 - val_loss: 2.0062 - val_f1: 0.0000e+00
Epoch 5/10
232000/232000 [==============================] - 180s 774us/step - loss: 1.6136 - f1: 0.4022 - val_loss: 2.0114 - val_f1: 2.4044e-04
Epoch 6/10
232000/232000 [==============================] - 180s 775us/step - loss: 1.6137 - f1: 0.4020 - val_loss: 2.0167 - val_f1: 1.3739e-04
Epoch 7/10
232000/232000 [==============================] - 175s 756us/step - loss: 1

In [52]:
data_test_1 = data_test.loc[:10,:]

In [187]:
y=np.array([data_test_1.loc[:,904:905].values , data_test_1.loc[:,906:907].values])

In [188]:
y.shape

(2, 10, 2)